<a href="https://colab.research.google.com/github/flavia-segadilha/Neuronio-Perceptron_2025_1/blob/main/NeuronioPerceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redes Neurais Artificiais – Projeto Prático 2.1
## Aprendizado Supervisionado no Neurônio Perceptron
Neste projeto prático, o objetivo é implementar o algoritmo de treinamento mediante Aprendizado Supervisionado do neurônio Perceptron de Rosenblatt aplicado em problemas de classificação.

---

### Equipe:


*   Adriana Raffaella Dos Santos Fonseca
*   Ana Flavia de Castro Segadilha da Silva
*   Guilherme Gonçalves Moraes
*   Yago De Oliveira Feitoza



## Bibliotecas

In [ ]:
#Importação das bibliotecas
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import math
from prettytable import PrettyTable
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

## Parte I – Resolvendo um Problema Linearmente Separável